#                      PYCITY SCHOOLS SNAPSHOT

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

### Total number of Schools, Students and Budget

In [2]:
# Get total schools
total_schools = len(school_data_complete["school_name"].unique())

# Get total students
total_students = school_data_complete["Student ID"].count()

# Get total budget
total_budget = school_data["budget"].sum()

### Calculating Scores

In [3]:
# Calculate the average math score
total_math_score = school_data_complete["math_score"].sum()
avg_math_score = total_math_score/total_students

# Calculate the average reading score
total_read_score = school_data_complete["reading_score"].sum()
avg_read_score = total_read_score/total_students

# Calculate the overall passing rate (overall average score), 
#i.e. (avg. math score + avg. reading score)/2
overal_passing_rate = (avg_math_score + avg_read_score)/2

### Calculating Passing (Score 70+) Percentages 

In [4]:
# Find out how many students pass & failed
math_scores = (school_data_complete["math_score"] >= 70).value_counts()

# Make df of scores, reset index of true and false, rename columns
math_scores_df = pd.DataFrame(math_scores).reset_index()
math_scores_rename = math_scores_df.rename(columns={"math_score":"student_count",
                                                   "index":"Passing Status: Math"})


# Filter for students who passed math and get the percentage
passed_math = math_scores_rename.iloc[0,:].sum()
percent_passed_math = passed_math*100/total_students

In [5]:
# Find out how many students pass & failed
reading_scores = (school_data_complete["reading_score"] >= 70).value_counts()
reading_scores

# Make df of scores, reset index of true and false, rename columns
read_scores_df = pd.DataFrame(reading_scores).reset_index()
read_scores_rename = read_scores_df.rename(columns={"reading_score":"student_count",
                                                   "index":"Passing Status: Reading"})

# Filter for students who passed reading and get the percentage
passed_read = read_scores_rename.iloc[0,:].sum()
percent_passed_read = passed_read*100/total_students

### Disctrict Summary DataFrame

In [6]:
# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],"Total Students": total_students,
                                    "Total Budget": [total_budget], "Average Math Score": avg_math_score,
                                   "Average Reading Score":avg_read_score, "Passing Math (%)": percent_passed_math,
                                   "Passing Reading(%)":percent_passed_read, 
                                   "Overall Passing Rate(%)":overal_passing_rate})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math (%),Passing Reading(%),Overall Passing Rate(%)
0,15,39170,24649428,78.985371,81.87784,74.983406,85.808016,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### School name and type

In [7]:
# Group by school name and find average
school_group = school_data_complete.groupby("school_name")
score_avg = school_group.mean()

# make size and budget as integers
size_bud_format = score_avg[["size", "budget"]].astype("int64")

# Merge df
merging_df = pd.merge(size_bud_format,score_avg, how="inner", on="school_name")
school_avg = merging_df[["size_x", "budget_x", "reading_score", "math_score"]]
schl_avg_rename = school_avg.rename(columns={"size_x": "Student Population", "budget_x":"Budget",
                                             "reading_score": "Average Reading Score",
                                            "math_score":"Average Math Score"})

# Merge type df to previously merged df and rename type
schl_type = school_data[["school_name","type"]]
merging_df2 = pd.merge(schl_type, schl_avg_rename, how="inner", on="school_name")
rename_schl = merging_df2.rename(columns={"type": "School Type"})

### School Budget

In [30]:
rename_schl["Budget per Student"]=""
rename_schl

# Calculate budget per student for each school
budget_student = rename_schl["Budget"]/rename_schl["Student Population"]

# Save budget per student in df and made a copy of df
rename_schl["Budget per Student"]=budget_student
df_copy = rename_schl.copy()
df_copy

,school_name,School Type,Student Population,Budget,Average Reading Score,Average Math Score,Budget per Student
0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
7,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
8,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


### Passing Score (70+) Percentage

In [9]:
# Calculate the percentage of students with a passing reading score (70 or greater)

# Select passing reading scores and group scores by school by count of students
read_pass = school_data_complete.loc[school_data_complete["reading_score"]>=70]
read_students = read_pass.groupby("school_name").count()

# Calculate the student count as a percentage and make a dataframe
school_pass_read = read_students["reading_score"]*100/score_avg["size"]
schl_pass_read_df = pd.DataFrame(school_pass_read)

# Rename dataframe to reflect percentage of students who passed
schl_pass_read_rename = schl_pass_read_df.rename(columns={0:"Passed Reading(%)"})

In [10]:
# Calculate the percentage of students with a passing math score (70 or greater)

# Select passing math scores and group scores by school by count of students
math_pass = school_data_complete.loc[school_data_complete["math_score"]>=70]
students_math = math_pass.groupby("school_name").count()

# Calculate the student count as a percentage and make a dataframe
school_pass_math = students_math["math_score"]*100/score_avg["size"]
schl_pass_math_df = pd.DataFrame(school_pass_math)

# Rename dataframe to reflect percentage of students who passed
schl_pass_math_rename = schl_pass_math_df.rename(columns={0:"Passed Math(%)"})

In [37]:
# Calculate the percentage of students with overall passing score
rate_df = (school_pass_math+school_pass_read)/2
rate_df = pd.DataFrame(rate_df)
overall_rate = rate_df.rename(columns={0:"Overall Rate(%)"})

### School Summary DataFrame

In [38]:
# Merge previous dataframes to create a summary and rename school name rename_schl
summary_df = pd.merge(df_copy, schl_pass_read_rename, how="left", on=["school_name"])
summary_df2 = pd.merge(summary_df, schl_pass_math_rename, how="left", on=["school_name"])
summary_df3 = pd.merge(summary_df2, overall_rate, how="left", on=["school_name"])
school_rename = summary_df3.rename(columns={"school_name": "School Name"})

# Reorganize df
school_summary_df = school_rename[["School Name","School Type","Student Population","Budget","Budget per Student",
                              "Average Reading Score","Average Math Score","Passed Reading(%)","Passed Math(%)",
                               "Overall Rate(%)"]]
school_summary_df

,School Name,School Type,Student Population,Budget,Budget per Student,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,95.854628,93.867121,94.860875
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [39]:
school_summary_df.sort_values("Overall Rate(%)", ascending=False).head()

,School Name,School Type,Student Population,Budget,Budget per Student,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,97.308869,93.272171,95.290520
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [40]:
school_summary_df.sort_values("Overall Rate(%)").head()

,School Name,School Type,Student Population,Budget,Budget per Student,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,80.220055,66.366592,73.293323
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,73.639992
13,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,73.804308


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
# Setting student id as the index
student_id = school_data_complete.set_index("Student ID")

In [ ]:
# student_id = school_data_complete.set_index("Student ID")
# student_id

# Filter scores by 9th grade
nine = student_id.loc[student_id["grade"]=="9th"]
nine_math_avg = nine.groupby("school_name").mean()
nine_math_avg_rename = nine_math_avg.rename(columns={"reading_score":"Grade 9 Reading Score",
                                                    "math_score":"Grade 9 Math Score"})

# Filter scores by 10th grade
ten = student_id.loc[student_id["grade"]=="10th"]
ten_math_avg = ten.groupby("school_name").mean()
ten_math_avg_rename = ten_math_avg.rename(columns={"reading_score":"Grade 10 Reading Score",
                            "math_score":"Grade 10 Math Score"})

# Filter scores by 11th grade
eleven = student_id.loc[student_id["grade"]=="11th"]
eleven_math_avg = eleven.groupby("school_name").mean()
eleven_math_avg_rename = eleven_math_avg.rename(columns={"reading_score":"Grade 11 Reading Score",
                                "math_score":"Grade 11 Math Score"})

# Filter scores by 12th grade
twelve = student_id.loc[student_id["grade"]=="12th"]
twelve_math_avg = twelve.groupby("school_name").mean()
twelve_math_avg_rename = twelve_math_avg.rename(columns={"reading_score":"Grade 12 Reading Score",
                                "math_score":"Grade 12 Math Score"})

In [ ]:
# Merging 9th & 10th grade
nine_ten_avg = pd.merge(nine_math_avg_rename, ten_math_avg_rename, how="left", on=["school_name"])

# Merging 11th & 12th grade
eleven_twelve_avg = pd.merge(eleven_math_avg_rename, twelve_math_avg_rename, how="left", on=["school_name"])

# Merging all grades
school_score_avg = pd.merge(nine_ten_avg, eleven_twelve_avg, how="left", on=["school_name"])

# Reset index and rename school name column
df_reset_index = school_score_avg.reset_index()
renaming_school = df_reset_index.rename(columns={"school_name":"School Name"})

In [ ]:
# Filter grades to get math scores per school
math_avg = renaming_school[["School Name","Grade 9 Math Score","Grade 10 Math Score",
                             "Grade 11 Math Score","Grade 12 Math Score"]]
math_avg

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
# Filter grades to get reading scores per school
reading_avg = renaming_school[["School Name","Grade 9 Reading Score","Grade 10 Reading Score",
                             "Grade 11 Reading Score","Grade 12 Reading Score"]]
reading_avg

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.